In [1]:
import os
import os.path as path
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la

from src.structured_random_features.src.models.weights import V1_weights

# Packages for fft and fitting data
from scipy import fftpack as fft
from sklearn.linear_model import Lasso

# Package for importing image representation
from PIL import Image, ImageOps


In [5]:
param = []

# Original Image size(1000, 662)
img_Original = Image.open("image/tree.jpg") 
img_Original = ImageOps.grayscale(img_Original)

# took 30 X 30 pixel part of tree image at about coord(650, 260)
img = Image.open("image/tree_part1.jpg")
img = ImageOps.grayscale(img)
imgArr = np.asarray(img)



# 1. Generate random V1 weights
numCell = np.random.randint(50, 2000)
dim = (100, 100)
n = 30


cellSize = np.random.uniform(0.01, 20)
sparseFreq = np.random.randint(1, 10)



# Store generated V1 cells in W
W = V1_weights(numCell, dim, cellSize, sparseFreq) 

    

W.shape

(145, 10000)

In [4]:

# Function: compress
# Parameters:
##     W: An opened index for measurement
##     y: the value of the opened index W
##     alpha: panelty value to fit for Lasso
##     dim (n, m): image size that needs to be reformed

# Return:
##     theta: matrix of W when FFT took in place
##     reformed: Reformed image in array
##     s: sparse vector s which is a estimated coefficient generated from LASSO
def compress(W, y, alpha, dim):
    n, m = dim
    smapleSz = W.shape[0]
    
    ## WΨ
    theta = fft.dctn(W.reshape(numCell, n, m), norm = 'ortho', axes = [1, 2])
    theta = theta.reshape(numCell, n*n)
    
    ## Initialize Lasso and Fit data
    mini = Lasso(alpha = alpha)
    mini.fit(theta, y)
    
    ## Retrieve sparse vector s
    s = mini.coef_
    
    # Reform the image using sparse vector s with inverse descrete cosine
    reform = fft.idctn(s.reshape(n, n), norm='ortho', axes=[0,1])
    
    #return theta, reformed img, sparse vectors
    return theta, reform, s

In [ ]:
## Return reform image array
    reformedImg = Image.fromarray(reform)
    reformedImg.save("reformedImage.png")